### Pt 1 LangGraph Basics: Building basic chatbot with no memory and no tools 
- Pt 1 Tutorial: [langgraph basics](https://langchain-ai.github.io/langgraph/tutorials/introduction/)

In [4]:
#general 
import os, sys, getpass
from typing import Annotated
from typing_extensions import TypedDict
from IPython.display import Image, display

#chat bot specific 
from anthropic import Anthropic
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.runnables import RunnableConfig


#Import API Keys        
_root = "/home/zjc1002/Mounts/code/admin/"
sys.path.append(_root)
from api_keys import _api_keys

#set Environment Variables 
def _set_env(var: str,value: str = None):
    if not os.environ.get(var):
        os.environ[var] = value

#SET ANTHROPIC API KEY ENV VAR
_set_env("ANTHROPIC_API_KEY"
         ,value =  _api_keys['ANTHROPIC_API_KEY'])

In [3]:
# Initialize the Anthropic client
client = Anthropic(api_key=_api_keys['ANTHROPIC_API_KEY'])

# Get available models
available_models = client.models.list()

# Print the available models
for model in available_models.data:
    print(f"Model ID: {model.id}")

    # The context_window property contains the maximum number of tokens
    if hasattr(model, 'context_window'):
        print(f"Max context length: {model.context_window} tokens")

del client , available_models

Model ID: claude-3-7-sonnet-20250219
Model ID: claude-3-5-sonnet-20241022
Model ID: claude-3-5-haiku-20241022
Model ID: claude-3-5-sonnet-20240620
Model ID: claude-3-haiku-20240307
Model ID: claude-3-opus-20240229
Model ID: claude-3-sonnet-20240229
Model ID: claude-2.1
Model ID: claude-2.0


### Select LLM TO  use in chatbot

In [6]:
#load model to use in chatbot (use tiny model that is cheap)
_model_id = "claude-3-haiku-20240307"

#### A. Create a ***StateGraph***. A StateGraph object defines the structure of our chatbot as a "state machine". 
  - **State** Consists of the schema of the graph as well as *reducer* functions which specify how to apply updates to the state. 
    - The schema of the State will be the input schema to all Nodes and Edges in the graph, and can be either a *TypedDict* or a *Pydantic* model. 
    - All Nodes will emit updates to the State which are then applied using the specified reducer function.

  - **Nodes**  Represent units of work/functions the llm and our chatbot can call. Nodes can contain 2 positional arugments. Note, node funcitons are converted to ***RunnableLambdas*** behind the scenes, adding batch and async support to the  node function 
    1) ***State:*** The schema of the graph*
    2) ***Config:*** dictionary containing optional configurable parameters the node can use (ex. thread_id, user info, etc..)*

    
  - **Edges**  specify how the bot should transition between these functions. *(Edges must be a TypedDict or Pydantic model)*. A node can have MULTIPLE edges. If a node has multiple out-going edges, ***all*** of those destination nodes are executed in parallel as part of the next superstep. There are 4 key types of edges 
    1) ***Normal Edges:*** go directly from one node to the n ext 
    2) ***Conditional Edges:*** call a funciton to determine which nodes to go to next 
    3) ***Entry Point:*** the node that is called first when user input arrivers 
    4) ***Conditional Entry Point:*** call a funciton to determine which nodes to call first when user input arrives 


***Note:*** *the Graph State includes the graphs SCHEMA and REDUCER FUNCTIONS  which handel state updates.* **add_messages()** is the reducer function used in this example to append new messages to the list instead of replacing them. Keys without a reducer annotation will overwrite previous values  

*Remember: Nodes do the work, edegtes tell what to do next*

In [7]:

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # in this case we use the reducer function add_messages to  append messages to the list, rather than overwriting them
    messages: Annotated[list, add_messages]

#define config schemea to use to pass configurable parameters in your API 
#as example i add a string paramter that will be prefixed to each message (not a good idea, but illistrates how to use config)
class ConifgSchema(TypedDict):
    prefix: str 


#basic state
# The state is a dictionary that contains the current state of the graph
# the config contains paramters you want to pass to the node operations
graph_builder = StateGraph(State, config_schema=ConifgSchema)


### Add a chatbot Node to the graph
llm = ChatAnthropic(model=_model_id)


#### B. Add a chatbot Node to StateGraph()
 - The **chatbot node** function takes the current State as input and returns a dictionary containing an updated messages list under the key "messages". **This is the basic pattern for all LangGraph node functions.**


In [8]:
# Note: the chatbot node function takes the current State as input and returns a dictionary containing an updated messages list under the key "messages". **This is the basic pattern for all LangGraph node functions.**
def chatbot(state: State, config: RunnableConfig)-> dict:
    prefix = config['configurable'].get("prefix","")
    _message = llm.invoke(state["messages"])
    
    return {"messages": [_message]}

In [9]:
# The first argument is the unique node name
# The second argument is the function or object that will be called whenever the node is used.
graph_builder.add_node("chatbot", chatbot)

#Next, add an entry point. This tells our graph where to start its work each time we run it.
graph_builder.add_edge(START, "chatbot")

#set an exit point. This tells the graph "any time this node is run, you can exit."
graph_builder.add_edge("chatbot", END)

# Finally, we compile the graph. This will check for any errors in the graph and prepare it for use. This creates a new class that we can use to run the graph.
# The compiled graph is a subclass of the original graph builder, so we can still use the same methods.
# The compiled graph will have a new name, so we can use it to run the graph.
# This will also create a new class that we can use to run the graph.
graph = graph_builder.compile() 

#display the graph
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

### Execute the compiled stateGraph (aka talk to the chatbot)

In [10]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        #import pdb;pdb.set_trace();
        
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)
            print(value["messages"][-1].response_metadata.keys())
            print(f"Message ID: {value['messages'][-1].response_metadata['id']}")
            print(f"Model Used: {value['messages'][-1].response_metadata['model']}")
            print(f"Chat Stop Reason: {value['messages'][-1].response_metadata['stop_reason']}")
            print(f"Chat Stop Sequence: {value['messages'][-1].response_metadata['stop_sequence']}")
            print(f"Chat usage: {value['messages'][-1].response_metadata['usage']}")

# This is a simple loop to get user input and stream the graph updates
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: Jack Johnson is a singer-songwriter, guitarist, and record producer. Here are some key facts about him:

- He was born in 1975 in Hawaii and is known for his laidback, acoustic folk-pop sound.

- Some of his most popular songs include "Sitting, Waiting, Wishing," "Better Together," "Upside Down," and "Good People."

- He rose to fame in the early 2000s with the release of his debut album Brushfire Fairytales in 2001, which went triple platinum.

- His music style is often described as "beach music" or "mellow rock" and features influences from folk, reggae, and blues.

- In addition to his music career, Johnson is also known for his environmental and social activism, supporting causes like sustainable energy and youth education.

- He has released a total of 7 studio albums, the most recent being 2017's All the Light Above It Too.

- Johnson has won several awards over his career, including a Grammy nomination and multiple Billboard Music Awards.

- He is considered one of t